In [4]:
import numpy as np
import pandas as pd
import plotly.express as px

from scipy.stats import ttest_ind
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import LinearSVC

import anndata
import numpy as np
import pandas as pd
import phate
import plotly.express as px
import scFates as scf
import statsmodels.api as sm

from scipy.stats import ttest_ind
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm



In [5]:
markers = [
    'Actin',
    'Bcat',
    'DAPI',
    'Btub',
    'CD44',
    'Ecadherin',
    'EpCAM',
    'Ncadherin',
    'pancytokeratin',
    'pSMAD23',
    'Vim',
    'ZEB1',
    'zo1'
 ]

In [ ]:
cell_df = pd.read_csv(r'./Cellprofiler outputs/exif_and_real.csv')
cell_df['condition'] = cell_df.Metadata_well.str[-1]
cell_df['row'] = cell_df.Metadata_well.str[0]

In [7]:
def get_correlations(df, group_names, pred_name):
    correlations = pd.DataFrame()
    groups = df.groupby(group_names)
    for feature in df.columns:
        if 'FileName' in feature or 'PathName' in feature or 'Metadata' in feature:
            pass
        elif f'_{pred_name}' in feature:
            pred_feature_multi = feature.replace(pred_name, pred_name, 1)
            real_feature = feature.replace(f'_{pred_name}', '', 1)


            pcorrelation = groups[[real_feature, pred_feature_multi]].corr('pearson').unstack().iloc[:,1].reset_index()
            pcorrelation.columns = group_names + ['pcc']
            pcorrelation['feature'] = feature
            pcorrelation['feature_cat'] = feature.split('_', 1)[0]
            pcorrelation['method'] = 'pearson'
            correlations = pd.concat([correlations, pcorrelation], axis=0)

    return correlations

In [8]:
correlations_td = get_correlations(cell_df, ['Metadata_fold'], 'td')
correlations_td['input'] = 'td'
correlations_td['marker'] = correlations_td['feature'].str.extract(r'_([^_]+)_td')

correlations_multi = get_correlations(cell_df, ['Metadata_fold'], 'multi')
correlations_multi['input'] = 'multi'
correlations_multi['marker'] = correlations_multi['feature'].str.extract(r'_([^_]+)_multi')

correlations = pd.concat([correlations_td, correlations_multi])

In [ ]:
correlation_per_cat = correlations[['feature_cat', 'input', 'pcc', 'marker']].groupby(['feature_cat', 'input', 'marker']).agg(np.median).reset_index()

fig = px.box(
    correlation_per_cat[correlation_per_cat['feature_cat'] != 'Location'],
    x='feature_cat',
    y='pcc',
    color='input',
    category_orders={'input': ['td', 'multi']},
    # facet_col='marker',
    # facet_col_wrap=7,
)

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_traces(boxpoints=False) 
fig.update_traces(boxmean=True)
fig.show()


In [ ]:
for f in ["Intensity", "RadialDistribution", "Texture"]:
    test_pop = correlation_per_cat[(correlation_per_cat['feature_cat']==f)]
    group1 = test_pop[test_pop['input']=='multi'].dropna()
    group2 = test_pop[test_pop['input']=='td'].dropna()

    #perform Welch's t-test
    test = ttest_ind(group1['pcc'], group2['pcc'], equal_var=False)
    print(f'{f} : {test.pvalue}')

In [11]:
results = {
    'f1':[],
    'precision':[],
    'recall':[],
    'fold': [],
    'experiment':[],
}

In [12]:
# Classification
def run_classification(df, features, name, results):

    cell_data = df
    
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    features_cell_df = cell_data[features].values

    fold = 0
    for train_index, test_index in skf.split(features_cell_df, cell_data.condition.astype('category')):
        fold = fold + 1
        X_train = features_cell_df[train_index]
        X_test = features_cell_df[test_index]
        Y_train = cell_data.condition.astype('category').values[train_index]
        Y_test = cell_data.condition.astype('category').values[test_index]

        scaler = StandardScaler()
        scaled_X_train = scaler.fit_transform(X_train)

        classifier = LinearSVC()
        classifier.fit(scaled_X_train, Y_train)

        scaled_X_test = scaler.fit_transform(X_test)
        y_hat = classifier.predict(scaled_X_test)

        class_report = classification_report(Y_test, y_hat, target_names=['cond1', 'cond2', 'cond3'], output_dict=True)
        f1 = class_report['weighted avg']['f1-score']
        precision = class_report['weighted avg']['precision']
        recall = class_report['weighted avg']['recall']

        results['fold'].append(fold)
        results['f1'].append(f1)
        results['precision'].append(precision)
        results['recall'].append(recall)
        results['experiment'].append(name)

    return results

In [13]:
# select relevant morphology features
morphology_features = [         
       'AreaShape_Area',
       'AreaShape_Compactness', 'AreaShape_Eccentricity',
       'AreaShape_EquivalentDiameter', 
       'AreaShape_Extent', 'AreaShape_FormFactor', 'AreaShape_MajorAxisLength',
       'AreaShape_MaxFeretDiameter', 'AreaShape_MaximumRadius',
       'AreaShape_MeanRadius', 'AreaShape_MedianRadius',
       'AreaShape_MinFeretDiameter', 'AreaShape_MinorAxisLength',
       'AreaShape_Perimeter', 'AreaShape_Solidity',
       'Neighbors_NumberOfNeighbors_Adjacent',
       'Neighbors_PercentTouching_Adjacent',
]

In [14]:
markers = [
    'Actin',
    'Bcat',
    'DAPI',
    'Btub',
    'CD44',
    'Ecadherin',
    'EpCAM',
    'Ncadherin',
    'pancytokeratin',
    'pSMAD23',
    'Vim',
    'ZEB1',
    'zo1'
 ]

In [15]:
# Common markers only

common_features = [col for col in cell_df.columns if (not '_multi' in col) and (not '_td' in col) and (not '_notd' in col)]
common_features = [col for col in common_features if (col.startswith('Intensity')) or (col.startswith('RadialDistribution')) or (col.startswith('Texture'))]
common_features = [col for col in common_features if ('DAPI' in col) or ('Actin' in col) or ('Bcat' in col)]

# results = run_classification(cell_df, common_features + morphology_features, 'relevant_common', results)

In [16]:
# TD label replacement
for m in markers:
    td_replacement_features = [col for col in cell_df.columns if ('_td' in col)]
    td_replacement_features = [col for col in td_replacement_features if (col.startswith('Intensity')) or (col.startswith('RadialDistribution')) or (col.startswith('Texture'))]
    td_replacement_features = [col for col in td_replacement_features if ('DAPI' in col) or ('Actin' in col) or ('Bcat' in col) or (m in col)]

    # results = run_classification(cell_df, td_replacement_features + morphology_features, f'label_replacement', results)

In [17]:
# real 4-plex features.

stdIF_class_df = pd.DataFrame()
for m in markers:
    real_features = [col for col in cell_df.columns if (not '_multi' in col) and (not '_td' in col) and (not '_notd' in col)]
    real_features = [col for col in real_features if (col.startswith('Intensity')) or (col.startswith('RadialDistribution')) or (col.startswith('Texture'))]
    real_features = [col for col in real_features if ('DAPI' in col) or ('Actin' in col) or ('Bcat' in col) or (m in col)]

    # results = run_classification(cell_df, real_features + morphology_features, f'stdIF', results)

In [18]:
# td exif
td_features = [col for col in cell_df.columns if ('_td' in col)] 
td_features = [col for col in td_features if (col.startswith('Intensity')) or (col.startswith('RadialDistribution')) or (col.startswith('Texture'))]  

# results = run_classification(cell_df, td_features + morphology_features, 'td_exif', results)

In [ ]:
# no TD exIF
fluorescence_exif_cells = pd.read_csv(r'./Cellprofiler outputs/fluorescence_exif.csv')

fluo_features = [col for col in fluorescence_exif_cells.columns if (('_notd' in col) or ('_DAPI' in col) or ('_Actin' in col) or ('_Bcat' in col)) and (not '_td' in col) and (not '_multi' in col)]
fluo_features = [col for col in fluo_features if (col.startswith('Intensity')) or (col.startswith('RadialDistribution')) or ((col.startswith('Texture')))]

# results = run_classification(fluorescence_exif_cells, fluo_features + morphology_features, 'fluorescence_exif', results)

In [23]:
# full exif features

multi_features = [col for col in cell_df.columns if (('_multi' in col) or ('_DAPI' in col) or ('_Actin' in col) or ('_Bcat' in col)) and (not '_td' in col) and (not '_notd' in col)]
multi_features = [col for col in multi_features if (col.startswith('Intensity')) or (col.startswith('RadialDistribution')) or ((col.startswith('Texture')))]
multi_features = multi_features 

# results = run_classification(cell_df, multi_features + morphology_features, 'Full Exif', results)

In [24]:
# real multiplexing
real_features = [col for col in cell_df.columns if (not '_multi' in col) and (not '_td' in col) and (not '_notd' in col)]
real_features = [col for col in real_features if (col.startswith('Intensity')) or (col.startswith('RadialDistribution')) or (col.startswith('Texture'))] 
# results = run_classification(cell_df, real_features + morphology_features, 'real multiplex', results)

In [ ]:
classification_df = pd.DataFrame.from_dict(results)

In [ ]:
fig = px.box(
    classification_df,
    x='experiment',
    y='f1',
)

fig.update_traces(boxpoints=False) 
fig.update_traces(boxmean=True)

fig

In [ ]:
# T-test between any of the experients. Here for example, comparison between common only and label replacement is shown
group1 = classification_df[(classification_df['experiment']=='relevant_common')]
group2 = classification_df[(classification_df['experiment']=='label_replacement')]

t = ttest_ind(group1['f1'], group2['f1'], equal_var=False)
print(f'common vs replacement {t.pvalue}')




In [ ]:
# manifold and trajectory analysis

In [91]:
def embed(df, reducer, features):
    scaler = StandardScaler()
    scaled_virtual = scaler.fit_transform(df[features])

    embedding = pd.DataFrame(reducer.fit_transform(scaled_virtual), columns=['x', 'y'])
    embedding = embedding.join(df.reset_index())
    embedding.condition = embedding.condition.astype(str)

    return embedding


# reducer = umap.TSNE()
# reducer = umap.UMAP()
reducer = phate.PHATE(knn=10, decay=20, t=4, n_jobs=-1, random_state=42)

In [ ]:
# common markers embedding
common_embedding = embed(cell_df, reducer, common_features)
px.scatter(
    common_embedding,
    x='x',
    y='y',
    color='condition'
    
)

In [ ]:
# Full ExIF embedding
exif_embedding = embed(cell_df, reducer, multi_features)
px.scatter(
    exif_embedding,
    x='x',
    y='y',
    color='condition',
    width=600,
    height=600
)

In [ ]:
# Experimentally multiplexed embedding
multiplexed_embedding = embed(cell_df, reducer, real_features)
px.scatter(
    multiplexed_embedding,
    x='x',
    y='y',
    color='condition'
    width=600,
    height=600
)

In [94]:
def run_pseudotime(df):

    df.condition = df.condition.astype(int)

    adata = anndata.AnnData(df[['x', 'y', 'condition']])
    adata.raw=adata

    adata.obsm['X_manifold'] = np.array(df[['x', 'y']])

    scf.tl.curve(adata,Nodes=8,use_rep="X_manifold",ndims_rep=2)
    scf.tl.convert_to_soft(adata,0.001,10000, n_steps=1)
    scf.pl.graph(adata,basis="manifold")
    # scf.tl.root(adata,"condition")
    scf.tl.root(adata, 'condition', tips_only=True)

    scf.tl.pseudotime(adata,n_jobs=-1,n_map=100,seed=42)

    scf.pl.trajectory(adata,basis="manifold",arrows=True,arrow_offset=3)

    df['ptime'] = np.array(adata.uns['pseudotime_list']['1']['t'])

    return df

In [ ]:
ptime_common = run_pseudotime(common_embedding)
ptime_exif = run_pseudotime(exif_embedding)
ptime_multiplexed = run_pseudotime(multiplexed_embedding)

In [ ]:
def lowess_smooth(groups):
    lresult = pd.DataFrame()
    for name, group in groups:
        x = group['pseudotime'].values
        y = group['value'].values
        lowess = sm.nonparametric.lowess
        smoothed = lowess(y, x, frac=0.2)  # frac controls the smoothing span
        lowess_results = pd.DataFrame(smoothed)
        lowess_results.columns = ['pseudotime', 'value']
        lowess_results['variable'] = group.iloc[0]['variable']
        # lowess_results['value'] = (lowess_results['value']-lowess_results['value'].min())/(lowess_results['value'].max()-lowess_results['value'].min())
        lresult = pd.concat([lresult, lowess_results])
    return lresult


def ptime_trend(df, feature='Intensity_MeanIntensity', suffix='', no_commons=False):
    
    if no_commons:
        f_cols = [f'{feature}_{x}{suffix}' for x in markers[3:]]
    else:
        f_cols = [f'{feature}_{x}{suffix}' for x in markers]

    tmp = df[['x','y','condition','ptime',] + f_cols]
    
    for col in f_cols:
        tmp[col] = (tmp[col] - tmp[col].mean()) / tmp[col].std()    

    melt = tmp.melt(id_vars=['x', 'y', 'condition', 'ptime'])

    melt['pseudotime'] = melt['ptime'].round(decimals=3)

    fig = px.line(
        lowess_smooth(melt.groupby(['pseudotime', 'variable']).agg(np.median).reset_index().groupby(['variable'])),

        x='pseudotime',
        y='value',
        color='variable'
    )
    
    return fig

In [ ]:
ptime_trend(ptime_common, suffix='', no_commons=False)

In [ ]:
# real experimentally labelled dynamics on the exif trajectory
ptime_trend(ptime_exif, suffix='', no_commons=False)


In [ ]:
# virtually labelled dynamics on the exif trajectory
ptime_trend(ptime_exif, suffix='_multi', no_commons=True)


In [ ]:
ptime_trend(ptime_multiplexed, suffix='', no_commons=False)